# BookRAG

## Установка библиотек

In [ ]:
!pip install streamlit streamlit-chat

In [ ]:
!pip install langchain

In [ ]:
!pip install -U langchain-community

In [ ]:
!pip install sentence-transformers

In [5]:
!pip install faiss-gpu

In [ ]:
!pip install openai

In [ ]:
!pip install langchain_openai

In [8]:
!pip install pypdf

In [ ]:
!pip install tiktoken

In [ ]:
!pip install ragas

## Препроцессинг данных

In [11]:
from langchain_community.document_loaders import PyPDFLoader
import re
from typing import List, Dict

In [12]:
# Путь к PDF файлу
PDF_FILE_PATH = '/content/dostoevskiy_bratya_karamazovy.pdf'

# Загрузка страниц из PDF
loader = PyPDFLoader(PDF_FILE_PATH)
pages = loader.load()[7:]

# # Обработка текста для каждой страницы
# for i in range(len(pages)):
#     # Текущий текст страницы
#     text = pages[i].page_content

#     # Убираем переносы слов (например, "припоминае-\nмого" -> "припоминаемого")
#     text = re.sub(r'-\n', '', text)

#     # Заменяем разрывы строк на пробелы (например, "\n" -> " ")
#     text = re.sub(r'\n', ' ', text)

#     # Удаляем лишние пробелы
#     text = re.sub(r'\s+', ' ', text).strip()

#     # Удаляем подстроку с названием книги и автором
#     text = re.sub(r'Ф\.?\s*М\.?\s*Достоевский\.?\s*«Братья Карамазовы»', '', text, flags=re.IGNORECASE)

#     # Перезаписываем текст страницы
#     pages[i].page_content = text

# Проверка количества страниц после обработки
total_pages = len(pages)
print(f"Всего страниц: {total_pages}")

Всего страниц: 503


In [13]:
pages[0]

Document(metadata={'source': '/content/dostoevskiy_bratya_karamazovy.pdf', 'page': 7}, page_content='Ф.  М.  Достоевский.  «Братья Карамазовы»\n8\n \nЧасть первая\n \n \nКнига первая\nИстория одной семейки\n \n \nI\nФедор Павлович Карамазов\n \nАлексей Федорович Карамазов был третьим сыном помещика нашего уезда Федора\nПавловича Карамазова, столь известного в свое время (да и теперь еще у нас припоминае-\nмого) по трагической и темной кончине своей, приключившейся ровно тринадцать лет назад\nи о которой сообщу в своем месте. Теперь же скажу об этом «помещике» (как его у нас назы-\nвали, хотя он всю жизнь совсем почти не жил в своем поместье) лишь то, что это был стран-\nный тип, довольно часто, однако, встречающийся, именно тип человека не только дрянного\nи развратного, но вместе с тем и бестолкового, – но из таких, однако, бестолковых, кото-\nрые умеют отлично обделывать свои имущественные делишки, и только, кажется, одни эти.\nФедор Павлович, например, начал почти что ни с чем, поме

In [14]:
def split_text_into_chunks_with_metadata(pages, chunk_size=500):
    current_book = None
    current_chapter = None
    current_chapter_title = None
    is_next_title = True

    chunks = []
    for page in pages:
        text = page.page_content
        page_number = page.metadata['page'] + 1

        # Поиск текущей книги, главы и названия
        for line in text.splitlines():
            book_match = re.match(r"^\s*Книга\s+(\w+)", line)
            chapter_match = re.match(r"^\s*([IVXLCDM]+)\s*$", line)

            if current_book and is_next_title:
                current_chapter_title = line.strip()
                is_next_title = False

            if book_match:
                current_book = f"Книга {book_match.group(1)}"

            if chapter_match:
                current_chapter = f"Глава {chapter_match.group(1)}"
                is_next_title = True

        # Разбиваем текст на чанки
        for i in range(0, len(text), chunk_size):
            chunk = text[i:i + chunk_size]

            # Убираем переносы слов (например, "припоминае-\nмого" -> "припоминаемого")
            chunk_text = re.sub(r'-\n', '', chunk)

            # Заменяем разрывы строк на пробелы (например, "\n" -> " ")
            chunk_text = re.sub(r'\n', ' ', chunk_text)

            # Удаляем лишние пробелы
            chunk_text = re.sub(r'\s+', ' ', chunk_text).strip()

            # Удаляем подстроку с названием книги и автором
            chunk_text = re.sub(r'Ф. М. Достоевский. «Братья Карамазовы»', '', chunk_text, flags=re.IGNORECASE)

            chunks.append({
                'chunk': f"Текст чанка {chunk_text}. | Источник: номер страницы {page_number}, {current_book}, {current_chapter}",
                'page': page_number,
                'book': current_book,
                'chapter': current_chapter,
                # 'chapter_title': current_chapter_title
            })
    return chunks

chunks = split_text_into_chunks_with_metadata(pages)
print(f"Всего чанков: {len(chunks)}")

Всего чанков: 3925


## Реализация RAG

In [15]:
import faiss
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

In [ ]:
api_key = input("Введите API ключ: ")

In [17]:
# Инициализация модели эмбеддингов
embeddings = OpenAIEmbeddings(model="text-embedding-3-large", openai_api_key=api_key)

# Создание списка текстов для индексации
texts = [chunk['chunk'] for chunk in chunks]
metadatas = [chunk for chunk in chunks]

# Создание векторного хранилища FAISS
vector_store = FAISS.from_texts(texts, embeddings, metadatas=metadatas)

# Сохранение индекса (опционально)
vector_store.save_local("faiss_index")

<ipython-input-17-70eeb91c0fe5>:2: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(model="text-embedding-3-large", openai_api_key=api_key)


In [18]:
llm = ChatOpenAI(
    model="gpt-4o",           # Specify the GPT-4o model
    temperature=0.00,         # Low creativity for precise answers
    openai_api_key=api_key,   # Your OpenAI API key
    max_tokens=2048           # Adjust as needed
)

In [19]:
from langchain.prompts import PromptTemplate

template = """Вы — ассистент, специализирующийся на предоставлении точных ответов на вопросы по книге "Братья Карамазовы" Фёдора Достоевского.

Используйте предоставленные отрывки из книги для формирования ответов. Обязательно:

1. Приводите полный и развернутый ответ на вопрос.
2. Затем приводите точные цитаты из текста книги, подтверждающие ваш ответ, извлекая их из контекста.
3. Указывайте источник информации.


Контекст:
{context}

Вопрос: {question}

Ответ:
1. [Развернутый и полный ответ на вопрос]
2. [Цитата из текста: "..."]
3. [Источник: страница page, book, chapter]
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["context", "question"]
)

In [20]:
# Создание цепочки Вопрос-Ответ с использованием RetrievalQA
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",  # Использование простого объединения
    retriever=vector_store.as_retriever(search_kwargs={"k": 5}),
    return_source_documents=True,  # Добавляем возврат источников
    chain_type_kwargs={"prompt": prompt}
)

In [21]:
def chatbot():
    print("Чат-бот по книге 'Братья Карамазовы'. Задайте свой вопрос или введите 'выход' для завершения.")
    while True:
        try:
            user_input = input("Вы: ")
            if user_input.lower() in ['выход', 'exit', 'quit']:
                print("Чат-бот: До свидания!")
                break
            response = qa_chain.invoke(user_input)
            # print(response["source_documents"])
            result = response['result']
            print(result)
        except Exception as e:
            print(f"Чат-бот: Извините, произошла ошибка: {e}")

if __name__ == "__main__":
    chatbot()

Чат-бот по книге 'Братья Карамазовы'. Задайте свой вопрос или введите 'выход' для завершения.
Вы: Кто является ключевым персонажем в книге?
Ответ:

1. Ключевым персонажем в книге "Братья Карамазовы" является Алеша Карамазов. Он играет центральную роль в повествовании, и его духовные поиски и внутренние переживания составляют важную часть сюжета. Алеша представлен как человек с глубокими моральными и духовными убеждениями, и его характер противопоставляется другим членам семьи Карамазовых, которые часто погружены в страсти и конфликты. Его путь к духовному просветлению и стремление к добру оказывают значительное влияние на развитие событий в романе.

2. Цитата из текста: "если бы не повлияло оно сильнейшим и известным образом на душу и сердце главного, хотя и будущего героя рассказа моего, Алеши, составив в душе его как бы перелом и переворот, потрясший, но и укрепивший его разум уже окончательно, на всю жизнь и к известной цели." 

3. Источник: номер страницы 218, Книга седьмая, Глава 

## Создание json файла с ответами на 60 вопросов для дальнейшей валидации метрик

In [29]:
# Список вопросов
queries = [
    # Описательные вопросы о персонажах
    "Какой характер у Фёдора Павловича Карамазова?",
    "Чем выделяется Алёша среди других братьев?",
    "Как Достоевский описывает Ивана Карамазова?",
    "Какие черты характера определяют Дмитрия Карамазова?",
    "Какова роль Смердякова в романе?",
    "Какими словами автор описывает Грушеньку?",
    "Как характеризуется Екатерина Ивановна?",
    "Как проявляется духовность Алёши в общении с другими людьми?",
    "Почему Фёдора Павловича считают отрицательным персонажем?",
    "Как раскрывается внутренний конфликт Ивана Карамазова?",
    "Какие черты Смердякова указывают на его тайные намерения?",
    "Как Грушенька меняется в течение романа?",
    "Как отношения между братьями Карамазовыми отражают их различия?",
    "Почему Екатерина Ивановна так предана Дмитрию?",
    "Как монастырь и старец Зосима влияют на Алёшу?",
    "Что символизирует старец Зосима для других персонажей?",
    "Какой образ создаёт автор для Ильи Снегирёва?",
    "Чем выделяется фигура Лизы в романе?",
    "Какую роль играет жена Снегирёва в развитии сюжета?",
    "Почему Дмитрия считают самым страстным из братьев?",

    # Вопросы о ключевых событиях
    "Почему Дмитрия обвиняют в убийстве отца?",
    "Какую роль играет эпизод с похищением денег?",
    "Что происходит в сцене встречи Алёши с Грушенькой?",
    "Почему Иван обсуждает с Алёшей легенду о Великом Инквизиторе?",
    "Как Алёша справляется с утратой старца Зосимы?",
    "Как Дмитрий оказывается в долгах?",
    "Почему Фёдор Павлович вызывает ненависть у своих детей?",
    "Как разворачивается сцена суда над Дмитрием?",
    "Какое значение имеет смерть Фёдора Павловича?",
    "Почему Смердяков решает убить Фёдора Павловича?",
    "Какую роль играет монастырь в жизни героев?",
    "Почему Иван сталкивается с моральным кризисом?",
    "Как проходят последние дни старца Зосимы?",
    "Что символизирует встреча Алёши с детьми?",
    "Как Грушенька раскрывает свои чувства к Дмитрию?",
    "Что происходит в сцене с золотыми червонцами?",
    "Как Дмитрий готовится к побегу?",
    "Как Иван узнаёт о планах Смердякова?",
    "Что символизирует поведение детей в эпизоде с похоронами Илюши?",
    "Какое влияние оказывает разговор Ивана с дьяволом?",

    # Вопросы на понимание мотивов персонажей
    "Почему Иван отрицает существование Бога?",
    "Какую цель преследует Смердяков, убивая Фёдора Павловича?",
    "Почему Дмитрий решается бежать, несмотря на риск?",
    "Зачем Алёша посвящает себя духовному пути?",
    "Почему Екатерина Ивановна продолжает защищать Дмитрия?",
    "Какие чувства движут Грушенькой по отношению к Дмитрию?",
    "Как Иван приходит к своей легенде о Великом Инквизиторе?",
    "Почему Алёша поддерживает братьев, несмотря на их недостатки?",
    "Как изменяются чувства Ивана после смерти отца?",
    "Что заставляет Дмитрия принять вину за преступление, которого он не совершал?",
    "Почему Смердяков верит, что Иван согласится с его поступком?",
    "Как Грушенька принимает решение остаться с Дмитрием?",
    "Зачем Екатерина Ивановна пытается отомстить Дмитрию?",
    "Почему Алёша решает вернуться в мир из монастыря?",
    "Как изменяется отношение Ивана к религии после событий романа?",
    "Почему Илюша так важен для Алёши?",
    "Какие мотивы скрываются за поведением Фёдора Павловича?",
    "Как страхи и сомнения Ивана влияют на его поступки?",
    "Почему Дмитрий так предан своей любви к Грушеньке?",
    "Какое значение имеет сцена покаяния Дмитрия перед Алёшей?"
]


examples = []

for query in queries:
    try:
        # Получение ответа от чат-бота
        response = qa_chain.invoke(query)
        generated_answer = response['result']
        source_documents = response.get("source_documents", [])
        context = "\n".join([doc.page_content for doc in source_documents])

        # Формирование примера для RAGAS
        examples.append({
            "query": query,
            "generated_answer": generated_answer,
            "context": context
        })
    except Exception as e:
        print(f"Ошибка при обработке запроса: {query}. {e}")

# Сохранение данных в файл
import json

with open("ragas_examples.json", "w", encoding="utf-8") as f:
    json.dump(examples, f, ensure_ascii=False, indent=4)

print("Данные успешно сохранены!")

Данные успешно сохранены!
